<img src="https://github.com/pmservice/ai-openscale-tutorials/raw/master/notebooks/images/banner.png" align="left" alt="banner">

# IBM Watson OpenScale: Model Deployment and Monitoring (Simplified)

This notebook deploys a German Credit Risk model to Watson Machine Learning (WML), onboards it to Watson OpenScale (WOS), configures monitors (quality, fairness, drift, explainability), and injects historical scoring and feedback data.

> **Runtime Requirement**: IBM Runtime 24.1 (Python 3.11 XS)  
> **Services Required**: Watson Machine Learning, Watson OpenScale, Cloud Object Storage  
> **Optional**: Db2 Warehouse or Databases for PostgreSQL for external datamart

Ensure all required credentials and configuration values are defined before running the cell below.


### Package installation


In [ ]:
# !rm -rf /home/spark/shared/user-libs/python3.7*

# If you are executing this notebook in non IBM Watson Studio env then uncomment the pip install statements
#!pip install --upgrade scikit-learn==1.1   
# !pip install --upgrade pandas==0.25.3 
# !pip install --upgrade requests==2.23 
# !pip install --upgrade numpy==1.20.3 
# !pip install --upgrade SciPy 

!pip install --upgrade ibm-watson-machine-learning
!pip install --upgrade "ibm-watson-openscale~=3.0.34" 

!rm german_credit_data_biased_training.csv
!rm german_credit_feedback_data.json
!rm german_credit_scoring_data.json

!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_data_biased_training.csv -O german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_feedback_data.json -O german_credit_feedback_data.json
!wget https://raw.githubusercontent.com/IBM/watson-openscale-samples/main/IBM%20Cloud/WML/assets/data/credit_risk/german_credit_scoring_data.json -O german_credit_scoring_data.json


## Step 1: Define Credentials and Basic Configuration




## Configuration Dictionary Inputs

The following table describes the inputs expected in the `config_dict` used to deploy and monitor a model. Some inputs are required, while others are optional and will default to predefined values if not provided.

| Input Name                  | Expected Type/Format     | Required/Optional | Default Value (if any)         | Description |
|----------------------------|--------------------------|-------------------|--------------------------------|-------------|
| `wml_space_id`             | `str` (UUID)             | Required          | –                              | ID of the Watson Machine Learning deployment space. |
| `bucket_name`              | `str`                    | Required          | –                              | Name of the Cloud Object Storage bucket. |
| `cloud_api_key`            | `str`                    | Required          | –                              | IBM Cloud API key used for authentication. |
| `cos_resource_crn`         | `str`                    | Required          | –                              | Cloud Object Storage resource CRN. |
| `cos_endpoint`             | `str` (URL)              | Required          | –                              | Endpoint URL for the COS instance. |
| `cos_api_key_id`           | `str`                    | Required          | –                              | API key ID for accessing COS. |
| `cos_api_key`              | `str`                    | Required          | –                              | API key for accessing COS . |
| `training_data_df`         | `pandas.DataFrame`       | Required          | –                              | DataFrame containing the training data. |
| `training_data_file_name`  | `str` (filename)         | Required          | –                              | Name of the training data file to upload to COS. |
| `model_object`             | Trained model object     | Required          | –                              | Trained model (e.g., scikit-learn pipeline) to be deployed. |



In [ ]:

CLOUD_API_KEY = "<EDIT THIS>"
SPACE_ID = "<EDIT THIS>"


TRAINING_DATA_FILE_NAME = "<EDIT THIS>" #eg "german_credit_data_biased_training.csv
COS_API_KEY_ID = "<EDIT THIS>"
COS_API_KEY = "<EDIT THIS>"
COS_RESOURCE_CRN = "<EDIT THIS>" # eg "crn:v1:bluemix:public:cloud-object-storage:global:a/3bf0d9003abfb5d29761c3e97696b71c:d6f04d83-6c4f-4a62-a165-696756d63903::"
COS_ENDPOINT = "<EDIT THIS>" # eg"https://s3.us-south.cloud-object-storage.appdomain.cloud"
BUCKET_NAME = "<EDIT THIS>"



## Step 2: Load and Upload Training Data

This section performs two key tasks:

1. **Load the training dataset** from a local CSV file into a Pandas DataFrame.
2. **Upload the dataset to Cloud Object Storage (COS)** so it can be referenced during model deployment and monitoring.


Ensure that the COS credentials and bucket name are correctly defined before running this step.


In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.linear_model import SGDClassifier
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance
import ibm_boto3
from ibm_botocore.client import Config


training_data_df = pd.read_csv(TRAINING_DATA_FILE_NAME)

cos_client = ibm_boto3.resource(
    "s3",
    ibm_api_key_id=CLOUD_API_KEY,
    ibm_service_instance_id=COS_RESOURCE_CRN,
    ibm_auth_endpoint="https://iam.bluemix.net/oidc/token",
    config=Config(signature_version="oauth"),
    endpoint_url=COS_ENDPOINT
)

with open(TRAINING_DATA_FILE_NAME, "rb") as file_data:
    cos_client.Object(BUCKET_NAME, TRAINING_DATA_FILE_NAME).upload_fileobj(Fileobj=file_data)


## Step 3: Train and Evaluate the Model

This section prepares and trains a machine learning pipeline using the German Credit Risk dataset. It also evaluates the model's performance using AUC (Area Under the ROC Curve).

The trained model is stored in `MODEL_OBJECT` and the full dataset in `DATA_DF` for later use in deployment and monitoring.


In [4]:
# Train model
train_data, test_data = train_test_split(training_data_df, test_size=0.2)
features_idx = np.s_[0:-1]
first_record_idx = np.s_[0]
string_fields = [type(fld) is str for fld in train_data.iloc[first_record_idx, features_idx]]

ct = ColumnTransformer([
    ("ohe", OneHotEncoder(), list(np.array(train_data.columns)[features_idx][string_fields]))
])

clf_linear = SGDClassifier(loss='log_loss', penalty='l2', max_iter=1000, tol=1e-5)

pipeline_linear = Pipeline([
    ('ct', ct), 
    ('clf_linear', clf_linear)
])

risk_model = pipeline_linear.fit(train_data.drop('Risk', axis=1), train_data.Risk)
MODEL_OBJECT = risk_model
TRAINING_DATA_DF = training_data_df

# Evaluate model
predictions = risk_model.predict(test_data.drop('Risk', axis=1))
indexed_preds = [0 if prediction == 'No Risk' else 1 for prediction in predictions]
real_observations = test_data.Risk.replace('Risk', 1).replace('No Risk', 0).values
auc = roc_auc_score(real_observations, indexed_preds)
print("Model AUC:", auc)


Model AUC: 0.7461949288838602


## Optional Configuration Inputs (Predefined Defaults)

The following inputs are optional. If not provided in the `config_dict`, they will default to predefined values specific to the German Credit Risk model.

| Input Name                  | Expected Type/Format     | Required/Optional | Default Value (if any)         | Description |
|----------------------------|--------------------------|-------------------|--------------------------------|-------------|
| `wml_url`                  | `str` (URL)              | Optional          | `"https://us-south.ml.cloud.ibm.com"` | URL of the Watson Machine Learning service. |
| `model_name`               | `str`                    | Optional          | `"Scikit German Risk Model WML V4"` | Name of the model asset. |
| `deployment_name`          | `str`                    | Optional          | `"Scikit German Risk Deployment WML V4"` | Name of the model deployment. |
| `label_field`              | `str`                    | Optional          | `"Risk"`                        | Target column in the dataset. |
| `asset_name`               | `str`                    | Optional          | Same as `model_name`           | Name of the model asset in OpenScale. |
| `db_credentials`           | `dict` or `None`         | Optional          | `None`                         | External database credentials (if used). |
| `schema_name`              | `str` or `None`          | Optional          | `None`                         | Schema name for external database. |
| `service_provider_name`    | `str`                    | Optional          | `"Watson Machine Learning V2"` | Name of the model provider. |
| `service_provider_description` | `str`               | Optional          | `"Added by tutorial WOS notebook."` | Description of the model provider. |
| `prediction_field`         | `str`                    | Optional          | `"prediction"`                 | Output field for predictions. |
| `probability_fields`       | `list[str]`              | Optional          | `["probability"]`              | List of probability output fields. |
| `feature_fields`           | `list[str]`              | Optional          | Predefined list of 20+ fields  | Input features used by the model. |
| `categorical_fields`       | `list[str]`              | Optional          | Subset of `feature_fields`     | Categorical input features. |
| `sample_data_fields`       | `list[str]`              | Optional          | Same as `feature_fields`       | Field names for sample scoring input. |
| `sample_data_values`       | `list[list]`             | Optional          | Two example records            | Sample records for scoring simulation. |
| `feedback_data_file`       | `str` (filename)         | Optional          | `"german_credit_feedback_data.json"` | File containing historical feedback records. |
| `scoring_data_file`        | `str` (filename)         | Optional          | `"german_credit_scoring_data.json"` | File containing historical scoring requests. |
| `quality_monitor_params`   | `dict`                   | Optional          | `{"min_feedback_data_size": 50}` | Parameters for quality monitor. |
| `quality_monitor_thr`      | `list[dict]`             | Optional          | AUC threshold at 0.80          | Threshold for triggering quality alerts. |
| `fairness_monitor_params`  | `dict`                   | Optional          | Predefined fairness config     | Configuration for fairness monitor. |
| `fairness_monitor_thr`     | `None` or `list[dict]`   | Optional          | `None`                         | Threshold for fairness alerts (if any). |
| `explainability_params`    | `dict`                   | Optional          | `{"enabled": True}`            | Enable or disable explainability. |
| `driftv2_params`           | `dict`                   | Optional          | Predefined drift config        | Drift monitor configuration. |
| `problem_type`             | `str`                    | Optional          | `"binary"`                     | Type of machine learning problem. Supported values include `"binary"`, `"multiclass"`, and `"regression"`.                           | Type of ML problem (e.g., binary, multiclass, regression). |
| `input_data_type`          | `str`                    | Optional          | `"structured"`                 | Type of input data. Supported values include `"structured"`, `"unstructured_image"`, `"unstructured_text"`, `"unstructured_audio"`, etc.                        | Type of input data (e.g., structured, unstructured). |
| `force_deployment_recreate`           | `bool`                   | Optional          | `False`         | Controls the Watson Machine Learning (WML) deployment. <br> - If `False`: The API will **reuse** an existing WML deployment if `deployment_name` is found. If not found, a new deployment is created.<br> - If `True`: The API will **delete** any existing WML deployment with `deployment_name` and then **create a new one** (resulting in a new internal Deployment ID).  |
| `monitor_new_version`           | `bool`                   | Optional          | `False`        |  Controls the Watson OpenScale monitoring subscription. <br> - If `False`: The API will **reuse** an existing OpenScale subscription if one is found for the current WML Deployment ID. If not found, a new subscription is created.<br> - If `True`: The API will **always create a brand new** OpenScale subscription, even if one already exists for the WML Deployment ID. |

### How force_deployment_recreate and monitor_new_version Combinations Affect Behavior:



The combination of these flags dictates the API's overall action:



1. **Default/Continue Run:**

* `force_deployment_recreate = False`

* `monitor_new_version = False`

* **Behavior:** Reuses existing WML Deployment and OpenScale Subscription if found. Performs initial data log and monitor run. 



2. **New Endpoint & New Monitoring:**

* `force_deployment_recreate = False`

* `monitor_new_version = True`

* **Behavior:** Creates a **new WML Deployment** (requires a new `deployment_name` input from the user). Creates a **new OpenScale Subscription**. Performs initial data log and monitor run for the new setup. 



3. **Overwrite Endpoint & New Monitoring:**

* `force_deployment_recreate = True`

* `monitor_new_version = True`

* **Behavior:** **Deletes** the existing WML Deployment and **creates a new one** with the same name. Creates a **new OpenScale Subscription**. Performs initial data log and monitor run for the new setup. 




Reference: Optional Parameter Configuration

For detailed steps on how to set and configure the optional parameters, please refer to the following notebook:

[Watson OpenScale and Watson ML Engine - scikit-learn Binary Classification Example](https://github.com/IBM/watson-openscale-samples/blob/main/IBM%20Cloud/WML/notebooks/binary/scikit_learn/Watson%20OpenScale%20and%20Watson%20ML%20Engine.ipynb)



## Step 4: Configure and Run Deployment & Monitoring

This section defines a single configuration dictionary (`config_dict`) that consolidates all required inputs for deploying the model, onboarding it to Watson OpenScale, and enabling monitors.


Once the configuration is complete, the `deploy_and_monitor_model()` function is called to:
1. Deploy the model to Watson Machine Learning.
2. Onboard it to Watson OpenScale.
3. Configure the selected monitors.
4. Inject historical scoring and feedback data.
5. Run the monitors.



In [5]:
# Configuration dictionary for deployment and monitoring
config_dict = {
    "wml_space_id": SPACE_ID,
    "bucket_name": BUCKET_NAME,
    "cloud_api_key": CLOUD_API_KEY,
    "cos_resource_crn": COS_RESOURCE_CRN,
    "cos_endpoint": COS_ENDPOINT,
    "cos_api_key_id": COS_API_KEY_ID,
    "cos_api_key": CLOUD_API_KEY,    
    "training_data_df": TRAINING_DATA_DF,
    "training_data_file_name": TRAINING_DATA_FILE_NAME,
    "model_object": MODEL_OBJECT,
}

# Execute deployment and monitoring pipeline
from ibm_watson_openscale.notebooks_simplification.predictive_models_simplification import deploy_and_monitor_model
deploy_and_monitor_model(config=config_dict)


Creating connections...
SUCCESS
Software Specification ID: 45f12dfe-aa78-5b8d-9f38-0ee223c47309
Storing model to WML ...
Storing model Done
Model ID: 90b888bd-73a4-4ab5-a88a-516ceacc22b5
Deploying model...


#######################################################################################

Synchronous deployment creation for uid: '90b888bd-73a4-4ab5-a88a-516ceacc22b5' started

#######################################################################################


initializing
Note: online_url and serving_urls are deprecated and will be removed in a future release. Use inference instead.

ready


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='af64cb82-d16d-4c30-8500-f6b5054e47b4'
------------------------------------------------------------------------------------------------


Deploying model completed successfully;  deployment uid = af64cb82-d16d-4c30-8500-f6b5054e47b4
D

WOS Data Mart,Data Mart created by WOS tutorial notebook,True,active,2025-06-29 03:58:59.694000+00:00,bf3506fb-a335-4b00-8c81-0b348b5a93f4


Using existing datamart bf3506fb-a335-4b00-8c81-0b348b5a93f4
Configuring service provider
Found existing WML service provider 'Watson Machine Learning V2' with ID: 0197bc7f-0e94-7bc3-8d84-af23452530a3
Configuring subscription



 Waiting for end of adding subscription 0197bfbc-5f83-7007-8803-d4e2c4e49e5b 




active

-------------------------------------------
 Successfully finished adding subscription 
-------------------------------------------


New subscription created successfully with the ID: 0197bfbc-5f83-7007-8803-d4e2c4e49e5b
Payload data set id: 0197bfbc-66b2-72af-b068-d9cd540586b2
Scoring the model to enable monitoring
scoring the model is completed
verifying that the payload logging records
Payload logging did not happen, performing explicit payload logging.
Number of records in the payload logging table: 20
Number of records in the payload logging table: 20
Creating quality monitor



 Waiting for end of monitor instance creation 0197bfbc-b044-7c8a-a059-10eee2e0f397 




p

2025-06-30 07:28:43.642000+00:00,true_positive_rate,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.36363636363636365,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,area_under_roc,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.6510489510489511,0.8,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,precision,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.75,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,matthews_correlation_coefficient,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.38627307784817894,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,f1_measure,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.48979591836734687,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,accuracy,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.7448979591836735,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,label_skew,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.6909336273400494,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,gini_coefficient,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.3020979020979022,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,log_loss,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.46745497257524876,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:28:43.642000+00:00,false_positive_rate,0197bfbc-fe7a-7042-98f0-1b780076bf84,0.06153846153846154,None,None,['model_type:original'],quality,0197bfbc-b044-7c8a-a059-10eee2e0f397,803acf58-ea6c-4a30-9122-76294d2943ea,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b


Note: First 10 records were displayed.
Creating fairness monitor



 Waiting for end of monitor instance creation 0197bfbd-6058-7d5c-b99a-e9913fa271c8 




active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


Running fairness monitor



 Waiting for end of monitoring run 1feb88bd-6e7b-4656-be35-a574c943e227 




error

-------------------------------
 Run failed with status: error 
-------------------------------


Reason: ['code: AIQFM6001, message: Payload logging table rows 20 are less than the minimum rows 100']
Monitor run details: {
    "result": "HTTP response",
    "headers": {
        "_store": {
            "date": [
                "Date",
                "Mon, 30 Jun 2025 07:29:38 GMT"
            ],
            "content-type": [
                "Content-Type",
                "application/json"
            ],
            "transfer-encoding": [
                "Transfer-Encoding",
                "




 Waiting for end of monitor instance creation 0197bfbe-16d2-7fd7-a85f-9065a8af96ef 




preparing...........
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


Running drift_v2 monitor



 Waiting for end of monitoring run eeab15d0-537e-40dd-affc-bad8940a15d0 




running........................................
finished

---------------------------
 Successfully finished run 
---------------------------


Monitor run details: {
    "result": "HTTP response",
    "headers": {
        "_store": {
            "date": [
                "Date",
                "Mon, 30 Jun 2025 07:35:13 GMT"
            ],
            "content-type": [
                "Content-Type",
                "application/json"
            ],
            "transfer-encoding": [
                "Transfer-Encoding",
                "chunked"
            ],
            "connection": [
                "Connection",
                "keep-aliv

2025-06-30 07:31:48.465811+00:00,records_processed,0197bfbf-d071-7962-a0fd-56be2bede486,20.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:31:48.465811+00:00,confidence_drift_score,0197bfbf-d071-7962-a0fd-56be2bede486,0.5448,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:31:48.465811+00:00,records_processed,0197bfbf-d071-7962-a0fd-56be2bede486,20.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:31:48.465811+00:00,confidence_drift_score,0197bfbf-d071-7962-a0fd-56be2bede486,0.5452,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for No Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:32:04.469134+00:00,records_processed,0197bfc0-0ef5-785a-992a-9ea74f17959d,20.0,None,None,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:32:04.469134+00:00,confidence_drift_score,0197bfc0-0ef5-785a-992a-9ea74f17959d,0.5313,None,0.05,"['algorithm_used:total_variation', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:32:04.469134+00:00,records_processed,0197bfc0-0ef5-785a-992a-9ea74f17959d,20.0,None,None,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:32:04.469134+00:00,confidence_drift_score,0197bfc0-0ef5-785a-992a-9ea74f17959d,0.5317,None,0.05,"['algorithm_used:overlap_coefficient', 'computed_on:payload', 'field_type:class', 'field_name:Class Probability for Risk']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:32:12.132703+00:00,records_processed,0197bfc0-2ce4-793d-80bb-ad98f95a7ef9,20.0,None,None,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b
2025-06-30 07:32:12.132703+00:00,prediction_drift_score,0197bfc0-2ce4-793d-80bb-ad98f95a7ef9,0.0249,None,0.05,"['algorithm_used:jensen_shannon', 'computed_on:payload', 'field_type:class', 'field_name:prediction']",drift_v2,0197bfbe-16d2-7fd7-a85f-9065a8af96ef,eeab15d0-537e-40dd-affc-bad8940a15d0,subscription,0197bfbc-5f83-7007-8803-d4e2c4e49e5b


Note: First 10 records were displayed.
Creating explainability monitor



 Waiting for end of monitor instance creation 0197bfc3-32bb-7bd2-9aff-bcbbbf01a8a3 




preparing
active

---------------------------------------
 Monitor instance successfully created 
---------------------------------------


Running explainability monitor for a sample production
Running explanations on scoring IDs: ['f56432c0-13d8-4213-ae61-3a46add4f76b-1']
{
  "metadata": {
    "explanation_task_ids": [
      "c22175bc-4324-4205-b1f0-c04b96637f19"
    ],
    "created_by": "IBMid-697000GE0W",
    "created_at": "2025-06-30T07:35:43.109193Z"
  }
}
